In [1]:
import os
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import dask.dataframe as dd
import socket
import struct
import random
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pickle
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
zepa = 'C:/Users/vanam/parquets' 
os.chdir(zepa)
os.listdir()

['.ipynb_checkpoints',
 '._exploratory3.ipynb',
 '._exploratory5.ipynb',
 '._labels_rand3_seed157.pkl',
 '._part.23.parquet',
 '._part.35.parquet',
 '._part.45.parquet',
 '._part.80.parquet',
 '._part.81.parquet',
 '._part.82.parquet',
 '._part.83.parquet',
 '._part.84.parquet',
 '._part.85.parquet',
 '._part.86.parquet',
 '._part.87.parquet',
 '._part.88.parquet',
 '._part.89.parquet',
 '._part.90.parquet',
 '._t_values_rand3_seed157.pkl',
 '._z_scores_rand10_seed157-2.pkl',
 '._z_scores_rand10_seed157.pkl',
 'exploratory3.ipynb',
 'exploratory5.ipynb',
 'exploratory7.ipynb',
 'labels_rand3_seed157.pkl',
 'part.23.parquet',
 'part.35.parquet',
 'part.45.parquet',
 'part.80.parquet',
 'part.81.parquet',
 'part.82.parquet',
 'part.83.parquet',
 'part.84.parquet',
 'part.85.parquet',
 'part.86.parquet',
 'part.87.parquet',
 'part.88.parquet',
 'part.89.parquet',
 'part.90.parquet',
 't_values_rand3_seed157.pkl',
 'z_scores_rand10_seed157-2.pkl',
 'z_scores_rand10_seed157.pkl']

In [6]:
output_filepath = "labels_rand3_seed157.pkl"

with open(output_filepath, 'rb') as f:
    labels_block = pickle.load(f)
labels_block.shape

(300000,)

In [8]:
output_filepath = "t_values_rand3_seed157.pkl"

with open(output_filepath, 'rb') as f:
    t_values = pickle.load(f)
t_values.shape

(300000, 37)

In [10]:
output_filepath = "z_scores_rand10_seed157.pkl"

with open(output_filepath, 'rb') as f:
    z_scores = pickle.load(f)
z_block = []
for p in range(300000):
    z_block.append(z_scores)
z_block = np.array(z_block)    
    
z_block.shape

(300000, 37)

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models,regularizers
from tensorflow.keras.layers import Input,Dense,concatenate
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt

In [12]:
def give_indices(data_size,fractions=(0.7,0.15,0.15)):

    train_size = int(data_size * fractions[0])
    val_size = int(data_size * fractions[1])
    test_size = int(data_size * fractions[2])


    train_indices = np.random.choice(data_size, size=train_size, replace=False)
    remaining_indices = np.setdiff1d(np.arange(data_size), train_indices)
    val_indices = np.random.choice(remaining_indices, size=val_size, replace=False)
    test_indices = np.setdiff1d(remaining_indices, val_indices)
    return (train_indices,val_indices,test_indices)

In [16]:
my_fractions = (0.8,0.1,0.1)
(tr,vl,te) = give_indices(len(labels_block),my_fractions)
(tr_z,val_z,te_z) = (np.array(z_block[tr]),np.array(z_block[vl]),np.array(z_block[te]))
(tr_t,val_t,te_t) = (np.array(t_values[tr]),np.array(t_values[vl]),np.array(t_values[te]))
(tr_l,val_l,te_l) = (np.array(labels_block[tr]),np.array(labels_block[vl]),np.array(labels_block[te]))

In [17]:
tr_t.shape,val_t.shape,te_t.shape

((240000, 37), (30000, 37), (30000, 37))

In [23]:
def give_model():
    input_shape_invariant = (37,) #(39,)
    input_shape_varying = (37,) #(39,)
    input_invariant = Input(shape=input_shape_invariant)
    input_varying = Input(shape=input_shape_varying)

    # branch for invariant part
    dense_invariant1 = Dense(64, activation='relu')(input_invariant)
    dense_invariant2 = Dense(128, activation='relu')(dense_invariant1)
    dense_invariant3 = Dense(256, activation='relu')(dense_invariant2)

    # branch for varying part
    dense_varying1 = Dense(64, activation='relu')(input_varying)
    dense_varying2 = Dense(128, activation='relu')(dense_varying1)
    dense_varying3 = Dense(256, activation='relu')(dense_varying2)
    dense_varying4 = Dense(256, activation='relu')(dense_varying3)
    dense_varying5 = Dense(256, activation='relu')(dense_varying4)

    concatenated = concatenate([dense_invariant3, dense_varying5])
    output1 = Dense(1024, activation='relu')(concatenated)
    output2 = Dense(512, activation='relu')(output1)
    output3 = Dense(256, activation='relu')(output2)
    #output = Dense(1, kernel_regularizer=regularizers.L2(0.01), activation='linear')(output3)

    output = Dense(1, kernel_regularizer=regularizers.L2(0.01), activation='sigmoid')(output3)

    #output = Dense(1, kernel_regularizer=regularizers.L2(0.01), activation='hard_sigmoid')(output3)
    #output = Dense(1, kernel_regularizer=regularizers.L2(0.01), activation='tanh')(output3)

    # Define the model
    model = Model(inputs=[input_invariant, input_varying], outputs=output)
    return model

In [24]:
model1 = give_model()
print(model1.summary())
#model1.compile(optimizer='adam',loss="hinge",metrics=['accuracy'])
model1.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])
#model1.compile(optimizer='adagrad',loss="binary_crossentropy",metrics=['accuracy'])
#model1.compile(optimizer='rmsprop',loss="binary_crossentropy",metrics=['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 37)]         0           []                               
                                                                                                  
 dense_15 (Dense)               (None, 64)           2432        ['input_4[0][0]']                
                                                                                                  
 input_3 (InputLayer)           [(None, 37)]         0           []                               
                                                                                                  
 dense_16 (Dense)               (None, 128)          8320        ['dense_15[0][0]']               
                                                                                            

In [25]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9755075899339890124
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14236516352
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14261773592195322809
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [29]:
EPOCHS = 200
batchSize = 64
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
history = model1.fit(
    x=[tr_z, tr_t],
    y=tr_l,
    batch_size=batchSize,
    epochs=EPOCHS,
    validation_data=([val_z, val_t], val_l),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
3750/3750 [==============================] - 14s 4ms/step - loss: 0.4840 - accuracy: 0.7509 - val_loss: 0.6312 - val_accuracy: 0.6552
Epoch 2/200
3750/3750 [==============================] - 15s 4ms/step - loss: 0.5461 - accuracy: 0.6876 - val_loss: 0.5132 - val_accuracy: 0.7206
Epoch 3/200
3750/3750 [==============================] - 14s 4ms/step - loss: 0.1841 - accuracy: 0.9347 - val_loss: 0.0979 - val_accuracy: 0.9705
Epoch 4/200
3750/3750 [==============================] - 27s 7ms/step - loss: 0.1325 - accuracy: 0.9618 - val_loss: 0.1085 - val_accuracy: 0.9697
Epoch 5/200
3750/3750 [==============================] - 13s 4ms/step - loss: 0.1046 - accuracy: 0.9686 - val_loss: 0.0857 - val_accuracy: 0.9747
Epoch 6/200
3750/3750 [==============================] - 26s 7ms/step - loss: 0.1046 - accuracy: 0.9674 - val_loss: 0.0795 - val_accuracy: 0.9764
Epoch 7/200
3750/3750 [==============================] - 32s 8ms/step - loss: 0.1006 - accuracy: 0.9698 - val_loss: 0.0858 -

In [31]:
output_filepath = "model1_10rand_sig_binxent_200epochearly.pkl"
model1.save(output_filepath)

from tensorflow.keras.models import load_model

INFO:tensorflow:Assets written to: model1_10rand_sig_binxent_200epochearly.pkl\assets


In [32]:
model2 = load_model(output_filepath)

In [33]:
predictions = model2.predict([te_z, te_t], verbose=1)

938/938 [==============================] - 2s 2ms/step


In [34]:
predicted_labels = (predictions > 0.5).astype(int)

In [35]:
from sklearn.metrics import classification_report
print(classification_report(te_l, predicted_labels))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97     10283
           1       0.98      0.98      0.98     19717

    accuracy                           0.98     30000
   macro avg       0.97      0.97      0.97     30000
weighted avg       0.98      0.98      0.98     30000



In [38]:
def give_model_s():
    #input_shape_invariant = (37,) #(39,)
    input_shape_varying = (37,) #(39,)
    #input_invariant = Input(shape=input_shape_invariant)
    input_varying = Input(shape=input_shape_varying)

    # branch for invariant part
    #dense_invariant1 = Dense(64, activation='relu')(input_invariant)
    #dense_invariant2 = Dense(128, activation='relu')(dense_invariant1)
    #dense_invariant3 = Dense(256, activation='relu')(dense_invariant2)

    # branch for varying part
    dense_varying1 = Dense(64, activation='relu')(input_varying)
    dense_varying2 = Dense(128, activation='relu')(dense_varying1)
    dense_varying3 = Dense(256, activation='relu')(dense_varying2)
    dense_varying4 = Dense(256, activation='relu')(dense_varying3)
    dense_varying5 = Dense(256, activation='relu')(dense_varying4)

    #concatenated = concatenate([dense_invariant3, dense_varying5])
    output1 = Dense(1024, activation='relu')(dense_varying5)
    output2 = Dense(512, activation='relu')(output1)
    output3 = Dense(256, activation='relu')(output2)
    #output = Dense(1, kernel_regularizer=regularizers.L2(0.01), activation='linear')(output3)

    output = Dense(1, kernel_regularizer=regularizers.L2(0.01), activation='sigmoid')(output3)

    #output = Dense(1, kernel_regularizer=regularizers.L2(0.01), activation='hard_sigmoid')(output3)
    #output = Dense(1, kernel_regularizer=regularizers.L2(0.01), activation='tanh')(output3)

    # Define the model
    model = Model(inputs=[input_varying], outputs=output)
    return model

In [39]:
model3 = give_model_s()
print(model3.summary())
model3.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 37)]              0         
                                                                 
 dense_33 (Dense)            (None, 64)                2432      
                                                                 
 dense_34 (Dense)            (None, 128)               8320      
                                                                 
 dense_35 (Dense)            (None, 256)               33024     
                                                                 
 dense_36 (Dense)            (None, 256)               65792     
                                                                 
 dense_37 (Dense)            (None, 256)               65792     
                                                                 
 dense_38 (Dense)            (None, 1024)              2631

In [41]:
EPOCHS = 200
batchSize = 64
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
history = model3.fit(
    x=[ tr_t],
    y=tr_l,
    batch_size=batchSize,
    epochs=EPOCHS,
    validation_data=([ val_t], val_l),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
3750/3750 [==============================] - 11s 3ms/step - loss: 0.5332 - accuracy: 0.7255 - val_loss: 0.4178 - val_accuracy: 0.7097
Epoch 2/200
3750/3750 [==============================] - 20s 5ms/step - loss: 0.4093 - accuracy: 0.7624 - val_loss: 0.3794 - val_accuracy: 0.8210
Epoch 3/200
3750/3750 [==============================] - 29s 8ms/step - loss: 0.3905 - accuracy: 0.7886 - val_loss: 0.3924 - val_accuracy: 0.7180
Epoch 4/200
3750/3750 [==============================] - 11s 3ms/step - loss: 0.3942 - accuracy: 0.7558 - val_loss: 0.3925 - val_accuracy: 0.7210
Epoch 5/200
3750/3750 [==============================] - 11s 3ms/step - loss: 0.3564 - accuracy: 0.8277 - val_loss: 0.3266 - val_accuracy: 0.9421
Epoch 6/200
3750/3750 [==============================] - 10s 3ms/step - loss: 0.2700 - accuracy: 0.9193 - val_loss: 0.2338 - val_accuracy: 0.9465
Epoch 7/200
3750/3750 [==============================] - 21s 6ms/step - loss: 0.2245 - accuracy: 0.9340 - val_loss: 0.1211 -

In [42]:
output_filepath = "model3_10rand_sig_binxent_200epochearly_simple.pkl"
model3.save(output_filepath)


INFO:tensorflow:Assets written to: model3_10rand_sig_binxent_200epochearly_simple.pkl\assets


In [43]:
model4 = load_model(output_filepath)

In [44]:
predictions = model4.predict([ te_t], verbose=1)

938/938 [==============================] - 1s 1ms/step


In [45]:
predicted_labels = (predictions > 0.5).astype(int)
print(classification_report(te_l, predicted_labels))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     10283
           1       0.98      0.98      0.98     19717

    accuracy                           0.98     30000
   macro avg       0.97      0.97      0.97     30000
weighted avg       0.98      0.98      0.98     30000

